# A Transformer-based recommendation system


In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
from torch.utils.data import DataLoader

In [3]:
df_train = pd.read_parquet("./artifacts/train_data.parquet")

In [4]:
df_train.head()

,user_id_index,movie_sequence,rating_sequence,sex,occupation_index,age_group_index
0,1,"[3118, 1010, 1673, 1251]","[0.75, 1.0, 0.75, 1.0]",1.0,11,1
1,1,"[1673, 1251, 2272, 1769]","[0.75, 1.0, 0.5, 1.0]",1.0,11,1
2,1,"[2272, 1769, 3340, 1190]","[0.5, 1.0, 0.75, 0.75]",1.0,11,1
3,1,"[3340, 1190, 2736, 258]","[0.75, 0.75, 1.0, 0.75]",1.0,11,1
4,1,"[2736, 258, 1177, 712]","[1.0, 0.75, 1.0, 0.5]",1.0,11,1


In [5]:
item = df_train.iloc[0]

In [6]:
item.to_dict()

{'user_id_index': 1,
 'movie_sequence': array([3118, 1010, 1673, 1251]),
 'rating_sequence': array([0.75, 1.  , 0.75, 1.  ]),
 'sex': 1.0,
 'occupation_index': 11,
 'age_group_index': 1}

In [7]:
%%writefile ./src/dataset.py
import torch
from torch.utils.data import Dataset
# from torch.nn.utils.rnn import pad_sequence


class RatingDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        
        item_dict = self.data.iloc[index].to_dict()
        item_dict['target_rating'] = item_dict['rating_sequence'][-1]
        item_dict['rating_sequence'] = item_dict['rating_sequence'][:-1]

        item_dict['target_movie']= item_dict['movie_sequence'][-1]
        item_dict['movie_sequence']= item_dict['movie_sequence'][:-1]


        sample = {k:torch.tensor(v,dtype=torch.long) for k,v in item_dict.items()}

        return sample

Overwriting ./src/dataset.py


In [8]:
from src.dataset import RatingDataset

In [9]:
train_dataset = RatingDataset(data=df_train) 

In [10]:
train_dataset[0]

{'user_id_index': tensor(1),
 'movie_sequence': tensor([3118, 1010, 1673]),
 'rating_sequence': tensor([0, 1, 0]),
 'sex': tensor(1),
 'occupation_index': tensor(11),
 'age_group_index': tensor(1),
 'target_rating': tensor(1),
 'target_movie': tensor(1251)}

In [11]:
loader = DataLoader(train_dataset,batch_size=4,shuffle=True)

In [12]:
for sample in loader:
    break


In [13]:
sample

{'user_id_index': tensor([ 752, 5912,  850, 4578]),
 'movie_sequence': tensor([[ 553,   29, 1160],
         [1284, 2330,  454],
         [1961, 1379, 2634],
         [2919,  316,  294]]),
 'rating_sequence': tensor([[1, 0, 1],
         [0, 0, 0],
         [0, 0, 0],
         [0, 1, 1]]),
 'sex': tensor([1, 0, 0, 0]),
 'occupation_index': tensor([4, 2, 1, 1]),
 'age_group_index': tensor([3, 3, 4, 3]),
 'target_rating': tensor([0, 0, 0, 0]),
 'target_movie': tensor([1336, 2850, 3775,  848])}